In [1]:
import csv
import psycopg2
import os

In [2]:
connection = psycopg2.connect(dbname="tracking", user="webapp", password="tesis2019")

In [3]:
carriers = ["Maersk", "Hapag-Lloyd", "Evergreen"]
databases = ["scraper2", "scraper3"]

In [4]:
def get_location_id(original_location):
    location = original_location.split("\n")[-1]
    cursor = connection.cursor()
    the_id = 0
    try:
        cursor.execute("SELECT id FROM tracking_location WHERE name = %s", (location,))
        result = cursor.fetchone()
        if result is None:
            print("Location {} not found...".format(location))
        else:
            the_id = result[0]
    finally:
        cursor.close()
        return the_id

In [9]:
def insert_container(data):
    cursor = connection.cursor()
    try:
        result = cursor.execute("INSERT INTO tracking_container (code, carrier_id, origin_id, destination_id, "
                                "processed) VALUES (%s, %s, %s, %s, %s)", data)
    finally:
        cursor.close()

In [6]:
def insert_containers(database, carrier):
    filename = "containers/{}-{}.csv".format(database, carrier)
    if os.path.exists(filename):
        with open(filename, "r") as file:
            reader = csv.DictReader(file)
            for row in reader:
                first_location = get_location_id(row["first_location"])
                last_location  = get_location_id(row["last_location"])
                if (first_location > 0) and (last_location > 0):
                    data = (row["container"], int(row["carrier"]), first_location, last_location, True)
                    insert_container(data)
    else:
        print("File {} does not exist".format(filename))
    # If everything goes ok, commit
    connection.commit()

In [12]:
insert_containers("scraper3", "Hapag-Lloyd")
insert_containers("scraper3", "Maersk")

In [13]:
connection.close()